In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import re
import os

In [2]:
# Análise Exploratória dos Dados

df = pd.read_csv('..\\data\\01_raw\\nps_mock.csv')

print("### Primeiras Linhas do Dataset")
display(df.head())

print("### Verificação de Valores Ausentes")
display(df.isnull().sum())

duplicatas = df.duplicated().sum()
print(f"Número de linhas duplicadas: {duplicatas}")


### Primeiras Linhas do Dataset


,Ocorrencia,Classificacao,Cliente,Documento,Data,Etapa,Relato
0,2001,Problema em saque,Ana Vitória Lima,954.813.602-33,2025-01-26,1,2025-01-26 - Cliente Ana Vitória Lima (954.813...
1,2001,Problema em saque,Ana Vitória Lima,954.813.602-33,2025-01-26,2,Cliente foi informado da resolução e recebeu d...
2,2001,Problema em saque,Ana Vitória Lima,954.813.602-33,2025-01-26,3,Cliente ameaça abrir reclamação no Banco Centr...
3,2002,Atendimento ruim,Maria Julia Fogaça,493.017.685-93,2025-03-17,1,2025-03-17 - Cliente Maria Julia Fogaça (493.0...
4,2002,Atendimento ruim,Maria Julia Fogaça,493.017.685-93,2025-03-17,2,Cliente ameaça abrir reclamação no Banco Centr...


### Verificação de Valores Ausentes


Ocorrencia       0
Classificacao    0
Cliente          0
Documento        0
Data             0
Etapa            0
Relato           2
dtype: int64

Número de linhas duplicadas: 1


In [3]:
# Remoção de linhas duplicadas
df = df.drop_duplicates()
print("### Após Remoção de Linhas Duplicadas")
print(f"Número de linhas duplicadas restantes: {df.duplicated().sum()}")

# Removendo linhas onde a coluna 'Relato' possui valores ausentes
df = df.dropna(subset=['Relato'])

print("Valores ausentes após o tratamento:")
display(df.isnull().sum())



### Após Remoção de Linhas Duplicadas
Número de linhas duplicadas restantes: 0
Valores ausentes após o tratamento:


Ocorrencia       0
Classificacao    0
Cliente          0
Documento        0
Data             0
Etapa            0
Relato           0
dtype: int64

In [4]:
def limpar_texto(texto):
    texto = str(texto).strip()  # Remove espaços extras no início/fim
    texto = re.sub(r"\s+", " ", texto)  # Remove múltiplos espaços
    texto = re.sub(r"[^\w\s,.!?-]", "", texto)  # Remove caracteres estranhos, mantendo pontuação
    return texto

df["Relato"] = df["Relato"].apply(limpar_texto)

In [9]:
# 4Agrupa os relatos por ocorrência, separando com "###"
df_grouped = df.groupby("Ocorrencia", as_index=False).agg(lambda x: "\n### ".join(x.astype(str)) if x.name == "Relato" else x.iloc[0])

df_grouped["Relato"]

0     2025-01-26 - Cliente Ana Vitória Lima 954.813....
1     2025-03-17 - Cliente Maria Julia Fogaça 493.01...
2     2025-02-25 - Cliente Dr. Gustavo Rezende 762.4...
3     2025-01-07 - Cliente Vitória Almeida 905.736.8...
4     2025-01-25 - Cliente Ana Sophia Lima 210.794.3...
5     2025-03-18 - Cliente Ana Lívia da Mota 637.158...
6     2025-03-20 - Cliente Emanuelly Carvalho 573.49...
7     2025-03-11 - Cliente Helena Campos 263.089.741...
8     2025-02-22 - Cliente João Lucas Dias 804.912.3...
9     2025-03-12 - Cliente João Pedro das Neves 691....
10    2025-02-08 - Cliente Olivia Fernandes 970.826....
11    2025-01-30 - Cliente Sr. Vitor Gabriel Porto 5...
12    2025-02-24 - Cliente Sophie Barros 257.089.346...
13    2025-02-10 - Cliente Gabrielly da Costa 506.42...
14    2025-02-15 - Cliente Caio Almeida 341.295.078-...
15    2025-01-12 - Cliente Maria Fernanda Novaes 816...
16    2025-02-05 - Cliente Vicente Correia 316.287.4...
17    2025-01-14 - Cliente Isaac Martins 815.906

In [10]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Função para criar o prompt
def criar_prompt(historico, classificacoes):
    """Cria o prompt para a classificação de reclamações."""
    categorias = "\n    - ".join(classificacoes)
    prompt = f"""
    Analise o seguinte histórico de atendimento e classifique o problema principal em uma das seguintes categorias:
    
    - {categorias}

    Cada atualização no histórico é separada por "###".

    Histórico:
    ### {historico}

    Responda apenas com o nome da categoria.
    """
    return prompt.strip()

# Função para classificar a ocorrência com base no histórico completo
def classificar_reclamacao(historico, classificacoes):
    """Classifica uma única ocorrência com base no histórico completo."""
    prompt = criar_prompt(historico, classificacoes)  # Chama a função para criar o prompt
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Lista de classificações
classificacoes = [
    'Problema em saque', 
    'Atendimento ruim', 
    'Fraude bancária', 
    'Cobrança indevida', 
    'Cartão clonado', 
    'PIX enviado errado', 
    'Problema com fatura do cartão', 
    'Problema com limite de crédito', 
    'Problema no acesso ao app', 
    'Débito não autorizado'
]

# Aplicando a função ao DataFrame
df_grouped["categoria"] = df_grouped["Relato"].apply(lambda relato: classificar_reclamacao(relato, classificacoes))

# 6️⃣ Salva os resultados no S3
output_file = "reclamacoes_classificadas.csv"
df_grouped.to_csv(output_file, index=False)